In [1]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
def read_tbl(file_path):
    df = pd.read_csv(file_path)
    return df

model = load_model('../../train/data/output/gist_classification_model.h5')
with open('../../train/data/output/tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

directory_path = "../../ner/data/output/"

def process_all_csv_files(directory_path):
    df = pd.DataFrame()

    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df_part = read_tbl(file_path)
            df_part = df_part[~((df_part.gist.fillna("") == ""))]
            df = pd.concat([df, df_part], ignore_index=True)
    
    return df

df = process_all_csv_files(directory_path)

texts = df['gist'].astype(str).tolist()

sequences = tokenizer.texts_to_sequences(texts)
maxlen = 100
data = pad_sequences(sequences, maxlen=maxlen)

predictions = model.predict(data)
predicted_labels = (predictions > 0.5).astype("int32")

df['relevant'] = predicted_labels

df

166/166 [==============================] - 0s 401us/step


,tracking_id,gist,relevant
0,2014-0864,Phone Complaint: Complainant stated Bronson Ge...,0
1,2014-0865,Officer Chaney informed Sgt. Glover that she a...,0
2,2014-0866,"The complainant stated on October 13, 2014, at...",0
3,2014-0867,Failed to attend PPEP class as instructed.,0
4,2014-0868,Complainant alleged that Officer Sentino faile...,0
...,...,...,...
5304,NaN,A prisoner who was arrested under item #K-1657...,0
5305,2017-0650,The complainant stated that Sgt. Gilliam refus...,1
5306,2017-0652,The complainant stated the officer allowed him...,0
5307,2017-0653,(District) Accused was ordered to see Dr. McDe...,0
